<a href="https://colab.research.google.com/github/mimichen123/dgmd-e17/blob/main/object-detection/Setup-Data-Collection/setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<iframe width="800" height="500"
src="https://www.youtube.com/embed/3jD02dxL6gg"
frameborder="0"
allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture"
allowfullscreen
style="margin: auto; display: block"></iframe>

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#clone your repository
!git clone https://github.com/mimichen123/dgmd-e17.git

Cloning into 'dgmd-e17'...
remote: Enumerating objects: 633, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 633 (delta 90), reused 27 (delta 27), pack-reused 520 (from 2)
Receiving objects: 100% (633/633), 6.27 MiB | 35.25 MiB/s, done.
Resolving deltas: 100% (156/156), done.


In [30]:
#change path to object-detection folder
import os
import sys

sys.path.append('/content/dgmd-e17/object-detection')

print(os.getcwd())
!ls /content


/content
dgmd-e17  sample_data


In [31]:
DATASET_DIR="/code/object-detection/assets/duckietown_object_detection_dataset"
IMAGE_SIZE = 416
# this is the percentage of real data that will go into the training set (as opposed to the testing set)
REAL_TRAIN_TEST_SPLIT_PERCENTAGE = 0.8

In [32]:
# Download a dataset of pre-labelled real images.
from utils.misc import runp

# download dataset
if not os.path.exists(DATASET_DIR):
    runp(f"rm -rf {DATASET_DIR}/*")
    runp(f"mkdir -p {DATASET_DIR}/images")
    runp(f"mkdir -p {DATASET_DIR}/labels")
    runp(f"mkdir -p {DATASET_DIR}/train/images")
    runp(f"mkdir -p {DATASET_DIR}/train/labels")
    runp(f"mkdir -p {DATASET_DIR}/val/images")
    runp(f"mkdir -p {DATASET_DIR}/val/labels")
else:
    print("Folder structure already exists!")

# download dataset
if not os.path.exists(f"{DATASET_DIR}/frames"):
    !wget -O /tmp/dataset.zip https://duckietown-public-storage.s3.amazonaws.com/assets/mooc/2022/duckietown_object_detection_dataset.zip
    runp(f"unzip -q /tmp/dataset.zip -d $(dirname {DATASET_DIR})")
    runp(f"rm /tmp/dataset.zip")
else:
    print("Dataset already downloaded!")

Folder structure already exists!
Dataset already downloaded!


In [33]:
#resize images and resize assoicatd bounding boxes
import json
import os
import cv2
import numpy as np
from tqdm import tqdm
from utils.misc import xminyminxmaxymax2xywfnormalized, train_test_split, makedirs, runp

with open(f"{DATASET_DIR}/annotation/final_anns.json") as anns:
    annotations = json.load(anns)

In [34]:
npz_index = 0

all_image_names = []

def save_img(img, boxes, classes):
    global npz_index
    cv2.imwrite(f"{DATASET_DIR}/images/real_{npz_index}.jpg", img)
    with open(f"{DATASET_DIR}/labels/real_{npz_index}.txt", "w") as f:
        for i in range(len(boxes)):
            f.write(f"{classes[i]} "+" ".join(map(str,boxes[i]))+"\n")
    npz_index += 1
    all_image_names.append(f"real_{npz_index}")

filenames = tqdm(os.listdir(f"{DATASET_DIR}/frames"))
for filename in filenames:
    img = cv2.imread(f"{DATASET_DIR}/frames/{filename}")

    orig_y, orig_x = img.shape[0], img.shape[1]
    scale_y, scale_x = IMAGE_SIZE/orig_y, IMAGE_SIZE/orig_x

    img = cv2.resize(img, (IMAGE_SIZE,IMAGE_SIZE))

    boxes = []
    classes = []

    if filename not in annotations:
        continue

    for detection in annotations[filename]:
        box = detection["bbox"]
        label = detection["cat_name"]

        if label not in ["duckie", "cone"]:
            continue

        orig_x_min, orig_y_min, orig_w, orig_h = box

        x_min = int(np.round(orig_x_min * scale_x))
        y_min = int(np.round(orig_y_min * scale_y))
        x_max = x_min + int(np.round(orig_w * scale_x))
        y_max = y_min + int(np.round(orig_h * scale_y))

        boxes.append([x_min, y_min, x_max, y_max])
        classes.append(1 if label == "duckie" else 2)

    if len(boxes) == 0:
        continue


    boxes = np.array([xminyminxmaxymax2xywfnormalized(box, IMAGE_SIZE) for box in boxes])
    classes = np.array(classes)-1

    save_img(img, boxes, classes)



train_test_split(all_image_names, REAL_TRAIN_TEST_SPLIT_PERCENTAGE, DATASET_DIR)

100%|██████████| 1963/1963 [00:31<00:00, 62.53it/s]


ALL IMAGE NAMES TO MOVE DURING THIS SPLIT: ['real_1', 'real_2', 'real_3', 'real_4', 'real_5', 'real_6', 'real_7', 'real_8', 'real_9', 'real_10', 'real_11', 'real_12', 'real_13', 'real_14', 'real_15', 'real_16', 'real_17', 'real_18', 'real_19', 'real_20', 'real_21', 'real_22', 'real_23', 'real_24', 'real_25', 'real_26', 'real_27', 'real_28', 'real_29', 'real_30', 'real_31', 'real_32', 'real_33', 'real_34', 'real_35', 'real_36', 'real_37', 'real_38', 'real_39', 'real_40', 'real_41', 'real_42', 'real_43', 'real_44', 'real_45', 'real_46', 'real_47', 'real_48', 'real_49', 'real_50', 'real_51', 'real_52', 'real_53', 'real_54', 'real_55', 'real_56', 'real_57', 'real_58', 'real_59', 'real_60', 'real_61', 'real_62', 'real_63', 'real_64', 'real_65', 'real_66', 'real_67', 'real_68', 'real_69', 'real_70', 'real_71', 'real_72', 'real_73', 'real_74', 'real_75', 'real_76', 'real_77', 'real_78', 'real_79', 'real_80', 'real_81', 'real_82', 'real_83', 'real_84', 'real_85', 'real_86', 'real_87', 'real_88

In [35]:
import os
import tempfile
import shutil
from typing import List
from datetime import datetime


def zip_sub_dirs(abs_root_dir: str, lst_rel_subdirs: List[str], output_basename: str) -> str:
    """Zip some sub-directories, return the zipped file's path"""

    # check no identical output file exists
    out_full = f"{output_basename}.zip"
    if os.path.exists(out_full):
        print(f"File already exists at: {out_full}")
        print("Rename/Move it to run.\nNo operations performed.")
        return ""

    # make temporary directory
    tmp_dir = tempfile.mkdtemp()
    print(f"[{datetime.now()}] Temporary directory created at: {tmp_dir}")

    # format subdir original and temporary paths
    original_paths = [os.path.join(abs_root_dir, _d) for _d in lst_rel_subdirs]
    tmp_paths = [os.path.join(tmp_dir, _d) for _d in lst_rel_subdirs]

    print(f"[{datetime.now()}] List of directories to include in the zip file:")
    # ensure all specified subdirs exist
    for subdir in original_paths:
        assert os.path.exists(subdir), f"Specified path does not exist: {subdir}\nAbort! No operations performed."
        print(f" - {subdir}")

    print(f"[{datetime.now()}] Move subdirs to the temp root dir")
    # move subdirs to the tmp dir
    for ori, tmp in zip(original_paths, tmp_paths):
        shutil.move(ori, tmp)

    # create the zip archive
    print(f"[{datetime.now()}] Compressing and creating the archive...")
    ret = shutil.make_archive(output_basename, 'zip', tmp_dir)

    print(f"[{datetime.now()}] Move subdirs back to original location")
    # move directories back to original location
    for tmp, ori in zip(tmp_paths, original_paths):
        shutil.move(tmp, ori)

    print(f"[{datetime.now()}] Finished. Archive created at: {ret}")
    return ret


# NOTE: DO NOT change these
# zip file basename for our dataset
ZIPPED_DATASET_BASENAME_FILE = "duckietown_object_detection_dataset"
# file/dir location constants
DATASET_DIR = "/code/object-detection/assets/duckietown_object_detection_dataset"
# path and file name (without file extension)
ZIPPED_DATASET_BASENAME_FULL = os.path.join(DATASET_DIR, ZIPPED_DATASET_BASENAME_FILE)
TRAIN_DIR = "train"
VALIDATION_DIR = "val"

_ = zip_sub_dirs(
    abs_root_dir=DATASET_DIR,
    lst_rel_subdirs=[TRAIN_DIR, VALIDATION_DIR],
    output_basename=ZIPPED_DATASET_BASENAME_FULL,
)


[2025-05-08 00:09:57.380322] Temporary directory created at: /tmp/tmpk64fq672
[2025-05-08 00:09:57.380444] List of directories to include in the zip file:
 - /code/object-detection/assets/duckietown_object_detection_dataset/train
 - /code/object-detection/assets/duckietown_object_detection_dataset/val
[2025-05-08 00:09:57.380512] Move subdirs to the temp root dir
[2025-05-08 00:09:57.380910] Compressing and creating the archive...
[2025-05-08 00:09:59.481721] Move subdirs back to original location
[2025-05-08 00:09:59.482209] Finished. Archive created at: /code/object-detection/assets/duckietown_object_detection_dataset/duckietown_object_detection_dataset.zip
